<a href="https://colab.research.google.com/github/krGaurav310/Projects/blob/main/Deep_Neural_network_On_Hospital_data_set.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#import libraries
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.metrics import cohen_kappa_score, make_scorer
from sklearn import preprocessing
import tensorflow as tf
from keras.models import Sequential
import pandas as pd
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow_addons as tfa
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD

In [ ]:
from google.colab import files
files.upload()

{}

In [ ]:
#load the train data and test data
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [ ]:
#Print first 5 rows
train.head()

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,patientid,City_Code_Patient,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,8,c,3,Z,3,radiotherapy,R,F,2.0,31397,7.0,Emergency,Extreme,2,51-60,4911,0-10
1,2,c,5,Z,2,radiotherapy,S,F,2.0,31397,7.0,Trauma,Extreme,2,51-60,5954,41-50
2,10,e,1,X,2,anesthesia,S,E,2.0,31397,7.0,Trauma,Extreme,2,51-60,4745,31-40
3,26,b,2,Y,2,radiotherapy,R,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,7272,41-50
4,26,b,2,Y,2,radiotherapy,S,D,2.0,31397,7.0,Trauma,Extreme,2,51-60,5558,41-50


In [ ]:
#Checking the number of rows and columns
train.shape

(244648, 17)

In [ ]:
#Checking for missing values in the train data
train.isnull().sum()

Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                             113
patientid                               0
City_Code_Patient                    3147
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [ ]:
#Checking for missing values in test data
test.isnull().sum()

Hospital_code                           0
Hospital_type_code                      0
City_Code_Hospital                      0
Hospital_region_code                    0
Available Extra Rooms in Hospital       0
Department                              0
Ward_Type                               0
Ward_Facility_Code                      0
Bed Grade                               0
patientid                               0
City_Code_Patient                    1385
Type of Admission                       0
Severity of Illness                     0
Visitors with Patient                   0
Age                                     0
Admission_Deposit                       0
Stay                                    0
dtype: int64

In [ ]:
#dropping patientid as it will not make any sense for prediction
#dropping city code patients as it has missing values in the test data as well, and imputing in test data or removing rows in test data is not advisable.
train=train.drop(["patientid","City_Code_Patient"],axis=1)
test=test.drop(["patientid","City_Code_Patient"],axis=1)

In [ ]:
#removing the other rows in train data with missing values
train=train.dropna()

In [ ]:
#dropping duplicate rows in train data
train=train.drop_duplicates()

In [ ]:
#Checking the classes in the target variable
train['Stay'].value_counts()

21-30                 68178
11-20                 57611
31-40                 43173
51-60                 28184
0-10                  16461
41-50                  8918
71-80                  8283
More than 100 Days     5203
81-90                  3929
91-100                 2245
61-70                  2128
Name: Stay, dtype: int64

In [ ]:
#grouping the classes in the target variable as mentioned in the problem statement
train['Stay'] = train['Stay'].replace(['0-10'],'0-20')
train['Stay'] = train['Stay'].replace(['11-20'],'0-20')
train['Stay'] = train['Stay'].replace(['21-30'],'21-40')
train['Stay'] = train['Stay'].replace(['31-40'],'21-40')
train['Stay'] = train['Stay'].replace(['41-50'],'41-60')
train['Stay'] = train['Stay'].replace(['51-60'],'41-60')
train['Stay'] = train['Stay'].replace(['61-70'],'61-80')
train['Stay'] = train['Stay'].replace(['71-80'],'61-80')
train['Stay'] = train['Stay'].replace(['81-90'],'More_than_80')
train['Stay'] = train['Stay'].replace(['91-100'],'More_than_80')
train['Stay'] = train['Stay'].replace(['More than 100 Days'],'More_than_80')

In [ ]:
#rechecking the target classes
train['Stay'].value_counts()

21-40           111351
0-20             74072
41-60            37102
More_than_80     11377
61-80            10411
Name: Stay, dtype: int64

In [ ]:
#label encoding the categorical variables
colnames = list(train.columns)
label_encoder = preprocessing.LabelEncoder() 
for col in colnames:
    if train[col].dtype==object:
        train[col]= label_encoder.fit_transform(train[col])

In [ ]:
#print top 5 rows after pre processing
train.head()

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,8,2,3,2,3,3,2,5,2.0,0,0,2,5,4911,0
1,2,2,5,2,2,3,3,5,2.0,1,0,2,5,5954,2
2,10,4,1,0,2,1,3,4,2.0,1,0,2,5,4745,1
3,26,1,2,1,2,3,2,3,2.0,1,0,2,5,7272,2
4,26,1,2,1,2,3,3,3,2.0,1,0,2,5,5558,2


In [ ]:
#Standardising the only numerical variable
train["Admission_Deposit"]= (train["Admission_Deposit"] - train["Admission_Deposit"].mean())/train["Admission_Deposit"].std()

In [ ]:
#Checking the final table before modelling
train.head(2)

,Hospital_code,Hospital_type_code,City_Code_Hospital,Hospital_region_code,Available Extra Rooms in Hospital,Department,Ward_Type,Ward_Facility_Code,Bed Grade,Type of Admission,Severity of Illness,Visitors with Patient,Age,Admission_Deposit,Stay
0,8,2,3,2,3,3,2,5,2.0,0,0,2,5,0.006086,0
1,2,2,5,2,2,3,3,5,2.0,1,0,2,5,0.979863,2


In [ ]:
#Checking the number of rows and columns
train.shape

(244313, 15)

In [ ]:
#Spliiting the independent and dependent variables
dataset = train.values
X = dataset[:,0:14].astype(float)
Y = dataset[:,14]

In [ ]:
#Creating dummy for the target classes
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
dummy_y = np_utils.to_categorical(encoded_Y)

In [ ]:
#Creating a loss metric Cohen Kappa using tensorflow addon
lossfunc=tfa.metrics.CohenKappa(num_classes=5,sparse_labels=False)

In [ ]:
#model development
model=Sequential([
                        Dense(14, input_shape=(14,),activation='relu'),
                        Dense(8,activation='relu'),
                        Dense(5,activation='softmax')
])
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=lossfunc
              )
model.fit(X,dummy_y,epochs=100)

Epoch 1/100
7635/7635 [==============================] - 10s 1ms/step - loss: 1.1580 - cohen_kappa: 0.1349
Epoch 2/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.1000 - cohen_kappa: 0.1941
Epoch 3/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.0885 - cohen_kappa: 0.2091
Epoch 4/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.0832 - cohen_kappa: 0.2151
Epoch 5/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.0809 - cohen_kappa: 0.2182
Epoch 6/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.0770 - cohen_kappa: 0.2228
Epoch 7/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.0749 - cohen_kappa: 0.2263
Epoch 8/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.0736 - cohen_kappa: 0.2266
Epoch 9/100
7635/7635 [==============================] - 9s 1ms/step - loss: 1.0729 - cohen_kappa: 0.2277
Epoch 10/100
7635/7635 [=====================

In [ ]:
#Checking the number of rows and columns in test data
test.shape

(73790, 15)

In [ ]:
#Grouping the target output classes as mentioned in the problem statement
test['Stay'] = test['Stay'].replace(['0-10'],'0-20')
test['Stay'] = test['Stay'].replace(['11-20'],'0-20')
test['Stay'] = test['Stay'].replace(['21-30'],'21-40')
test['Stay'] = test['Stay'].replace(['31-40'],'21-40')
test['Stay'] = test['Stay'].replace(['41-50'],'41-60')
test['Stay'] = test['Stay'].replace(['51-60'],'41-60')
test['Stay'] = test['Stay'].replace(['61-70'],'61-80')
test['Stay'] = test['Stay'].replace(['71-80'],'61-80')
test['Stay'] = test['Stay'].replace(['81-90'],'More_than_80')
test['Stay'] = test['Stay'].replace(['91-100'],'More_than_80')
test['Stay'] = test['Stay'].replace(['More than 100 Days'],'More_than_80')

In [ ]:
#Rechecking the target output classes
test["Stay"].value_counts()

21-40           31137
0-20            27576
41-60            9613
More_than_80     2893
61-80            2571
Name: Stay, dtype: int64

In [ ]:
#Label encoding the categorical variables
colnames = list(test.columns)
label_encoder = preprocessing.LabelEncoder() 
for col in colnames:
    if test[col].dtype==object:
        test[col]= label_encoder.fit_transform(test[col])

In [ ]:
#Standardising the numerical variable
test["Admission_Deposit"]= (test["Admission_Deposit"] - test["Admission_Deposit"].mean())/test["Admission_Deposit"].std()

In [ ]:
#Splitting data into depedent and independent variables
dataset = test.values
X_test = dataset[:,0:14].astype(float)
Y_test = dataset[:,14]

In [ ]:
#Creating dummy for the target classes
encoder = LabelEncoder()
encoder.fit(Y_test)
encoded_Y = encoder.transform(Y_test)
dummy_ytest = np_utils.to_categorical(encoded_Y)

In [ ]:
#Evaluating the test data
score = model.evaluate(X_test, dummy_ytest)

2306/2306 [==============================] - 2s 979us/step - loss: 1.0647 - cohen_kappa: 0.2416
